In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.ml.recommendation import ALS

In [ ]:
# Build the SparkSession
spark = SparkSession.builder \
   .master("local[*]") \
   .appName("Nome do Projeto") \
   .config("spark.executor.memory", "6gb") \
   .config('spark.sql.debug.maxToStringFields', 2000) \
   .config('spark.debug.maxToStringFields', 2000) \
   .config("spark.sql.caseSensitive", "false") \
   .getOrCreate()
   
sc = spark.spark

In [ ]:
# Carrega os dados no formato ALS (user, item, rating)
ratingsRDD = sc.textFile("user-item.txt")
ratingsRDD.collect()

In [ ]:
# Convertendo as strings
ratingsRDD2 = ratingsRDD.map(lambda l: l.split(',')).map(lambda l:(int(l[0]), int(l[1]), float(l[2])))

In [ ]:
# Criando um Dataframe
ratingsDF = spSession.createDataFrame(ratingsRDD2, ["user", "item", "rating"])

In [ ]:
ratingsDF.show()

In [ ]:
# Construindo o modelo
# ALS = Alternating Least Squares --> Algoritmo para sistema de recomendação, que otimiza a loss function 
# e funciona muito bem em ambientes paralelizados
als = ALS(rank = 10, maxIter = 5)
modelo = als.fit(ratingsDF)

In [ ]:
# Visualizando o Affinity Score
modelo.userFactors.orderBy("id").collect()

In [ ]:
# Criando um dataset de teste com usuários e items para rating
testeDF = spSession.createDataFrame([(1001, 9003),(1001,9004),(1001,9005)], ["user", "item"])

In [ ]:
# Previsões  
# Quanto maior o Affinity Score, maior a probabilidade do usuário aceitar uma recomendação
previsoes = (modelo.transform(testeDF).collect())
previsoes